In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, length, split

# to install nltk into databricks cluster, go to the cluster > libraries and add nltk
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

# Creation of the spark session
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
sc = spark._sc
sc = spark.sparkContext




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Unzipping tokenizers/punkt.zip.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.

In [0]:
# Loading the data into a RDD to better use spark functions
data = sc.textFile("/FileStore/tables/sherlock.txt").flatMap(lambda line: line.split(" "))

# Tokenizing the data to have each word as an element
words = data.flatMap(word_tokenize)

# Setting the stopwords and punctuations to be removed from the data
stop_words = set(stopwords.words('english') +  [",", ".", "“", "”", "’", "?", "‘", "!", ";"])

# Removing all stopwords and punctuations from the data
book_words = words.filter(lambda x: x.lower() not in stop_words)

# Creating the dictionary with all the words and their respective count, sorted in descending order
wordCounts = book_words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b).sortBy(lambda a:a[1], ascending=False)

# Saving final data into file as parquet (coalesce(1) is to return only one parquet file)
wordCounts.coalesce(1).saveAsTextFile("/FileStore/teste/resultado2")